In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rd
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I

In [2]:
m = ModelIntv()
n_collect = 200

In [3]:
out_path = f'out/dy_ladd'
pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

In [4]:
for p in pars_post:
    p.update(r_acf)

In [5]:
results = []


for i, pars in enumerate(pars_post[:n_collect]):
    y1, pars = m.find_baseline(pars, 2022)

    _, ms00, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 0}, 'MDU': {'Scale': 0}}, t_end=2031)
    _, ms01, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 0}, 'MDU': {'Scale': 1}}, t_end=2031)
    _, ms10, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 1}, 'MDU': {'Scale': 0}}, t_end=2031)
    _, ms11, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 1}, 'MDU': {'Scale': 1}}, t_end=2031)
    
    inc00 = (ms00.IncR * ms00.Pop).sum() * 0.5
    inc01 = (ms01.IncR * ms01.Pop).sum() * 0.5
    inc10 = (ms10.IncR * ms10.Pop).sum() * 0.5
    inc11 = (ms11.IncR * ms11.Pop).sum() * 0.5
    
    results.append({
        'Key': i,
        'Avt_MDU': 1 - inc01 / inc00,
        'Avt_D2D': 1 - inc10 / inc00,
        'Avt_MDU+D2D': 1 - inc11 / inc00,
    })
    
results = pd.DataFrame(results)

In [6]:
results.describe()

,Key,Avt_MDU,Avt_D2D,Avt_MDU+D2D
count,200.000000,200.000000,200.000000,200.000000
mean,99.500000,0.006060,0.001944,0.007973
std,57.879185,0.002975,0.001064,0.003859
min,0.000000,0.001963,0.000497,0.002458
25%,49.750000,0.003745,0.001259,0.005167
50%,99.500000,0.005449,0.001721,0.007226
75%,149.250000,0.007498,0.002371,0.009697
max,199.000000,0.019772,0.008067,0.027487


In [7]:
results.to_csv(f'{out_path}/Sim_BgACF_IncrementalAverted.csv')

In [8]:
[c for c in ms00.columns if c.startswith('ACF_D2D') or c.startswith('ACF_MDU')]

['ACF_MDU_Footfall',
 'ACF_MDU_Screened',
 'ACF_MDU_Confirmed',
 'ACF_MDU_Sym',
 'ACF_MDU_Vul',
 'ACF_MDU_CXR',
 'ACF_MDU_Xpert',
 'ACF_MDU_Yield',
 'ACF_MDU_TP',
 'ACF_MDU_DS_Fl',
 'ACF_MDU_DR_Fl',
 'ACF_MDU_DR_Sl',
 'ACF_D2D_Footfall',
 'ACF_D2D_Screened',
 'ACF_D2D_Confirmed',
 'ACF_D2D_Sym',
 'ACF_D2D_Vul',
 'ACF_D2D_CXR',
 'ACF_D2D_Xpert',
 'ACF_D2D_Yield',
 'ACF_D2D_TP',
 'ACF_D2D_DS_Fl',
 'ACF_D2D_DR_Fl',
 'ACF_D2D_DR_Sl']

In [9]:
idx = ['Pop', 'IncR', 'MorR'] + [c for c in ms00.columns if c.startswith('ACF_D2D') or c.startswith('ACF_MDU')]

results = []


for i, pars in enumerate(pars_post[:n_collect]):
    y1, pars = m.find_baseline(pars, 2022)

    _, ms0, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 0}, 'MDU': {'Scale': 0}}, t_end=2031)
    _, ms1, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 1}, 'MDU': {'Scale': 1}}, t_end=2031)
    _, ms2, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 2}, 'MDU': {'Scale': 2}}, t_end=2031)
    _, ms4, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 4}, 'MDU': {'Scale': 4}}, t_end=2031)

    results.append(pd.concat([
        ms0[idx].reset_index().assign(Key=i, Scale='Baseline'),
        ms1[idx].reset_index().assign(Key=i, Scale='1x'),
        ms2[idx].reset_index().assign(Key=i, Scale='2x'),
        ms4[idx].reset_index().assign(Key=i, Scale='4x')
    ]))

results = pd.concat(results)

In [10]:
results.head()

,Time,Pop,IncR,MorR,ACF_MDU_Footfall,ACF_MDU_Screened,ACF_MDU_Confirmed,ACF_MDU_Sym,ACF_MDU_Vul,ACF_MDU_CXR,...,ACF_D2D_Vul,ACF_D2D_CXR,ACF_D2D_Xpert,ACF_D2D_Yield,ACF_D2D_TP,ACF_D2D_DS_Fl,ACF_D2D_DR_Fl,ACF_D2D_DR_Sl,Key,Scale
0,2022.0,2.383217,0.010518,0.000815,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Baseline
1,2022.5,2.403200,0.010522,0.000815,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Baseline
2,2023.0,2.423351,0.010526,0.000816,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Baseline
3,2023.5,2.443670,0.010530,0.000816,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Baseline
4,2024.0,2.464160,0.010534,0.000816,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Baseline


In [11]:
results.to_csv(f'{out_path}/Sim_BgACF_ScaleUp.csv')